In [1]:
import altair as alt
import numpy as np
import pandas as pd

In [16]:
# preprocess data (this data has Age population percentage 10-14, male, but no same age range for female.. )
health_data_url = "https://raw.githubusercontent.com/ZeningQu/World-Bank-Data-by-Indicators/master/health/health.csv"
# downloaded from https://data.worldbank.org/indicator/SP.POP.0014.TO.ZS?view=chart and convert to readable csv
population_distribution_file = "API_SP.POP.0014.TO.ZS_DS2_en_csv_v2_1500178/API_SP.POP.0014.TO.ZS_DS2_en_csv_v2_1500178 (copy).csv"

# regions_url = "https://raw.githubusercontent.com/ZeningQu/World-Bank-Data-by-Indicators/master/health/Metadata_Country_API_8_DS2_en_csv_v2_10138079.csv"

In [17]:
def load_data(source):
    data = pd.read_csv(source, header=0, skipinitialspace=True)
    return data

In [18]:
health_df = load_data(health_data_url)

In [19]:
# Not sure how this happens, but there are some rows with sum of age_intervals frac by gender is larger than hundred percent
# weird indeed considering the age range 0-14 is not even included
age_ranges = ['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80 and above']
drop_cols = []
for idx, row in health_df.iterrows():
    male_sum = 0
    for f in age_ranges:
        male_sum += row['Population ages {}, male (% of male population)'.format(f)]
    if male_sum > 100:
        drop_cols.append(idx)
        continue
    female_sum = 0
    for f in age_ranges:
        female_sum += row['Population ages {}, female (% of female population)'.format(f)]
    if female_sum > 100:
        drop_cols.append(idx)
print(len(drop_cols))        
health_df = health_df.drop(health_df.index[drop_cols]) 

952


In [20]:

# remove regions, only keep countries
names_exclude = ['Sub-Saharan Africa', 'IDA', 'income', 'Fragile and conflict affected situations', 'IBRD',
                 'poor countries', 'demographic', 'developed countries: UN classification', 'Europe', 'Asia',
                 'Middle East', 'Pacific', 'North America', 'small states', 'members', 'World', 'Small states',  'Latin America & Caribbean']
for name in names_exclude:
    health_df = health_df[~health_df['Country Name'].str.contains(name)]
#print(health_df['Country Name'].unique())    

In [21]:
# drop unused columns
health_df = health_df.drop(columns=["Country Code", "Population growth (annual %)", 'Population ages 15-64 (% of total)', 'Population ages 15-64, female',
       'Population ages 15-64, female (% of total)',
       'Population ages 15-64, male',
       'Population ages 15-64, male (% of total)', 'Population ages 65 and above (% of total)',
       'Population ages 65 and above, female',
       'Population ages 65 and above, female (% of total)',
       'Population ages 65 and above, male',
       'Population ages 65 and above, male (% of total)',
       'Population ages 10-14, male (% of male population)',
       'Population ages 65 and above, total', "Sex ratio at birth (male births per female births)"])

In [22]:
missing_df = load_data(population_distribution_file)
# add missing column to health data
missing_col = missing_df.head(1)['Indicator Name'].item()
health_df[missing_col] = np.nan

In [23]:
years = list(missing_df.columns)[4:-1]
years = [int(i) for i in years] 
for index, row in missing_df.iterrows():
    country_name = row['Country Name']
    matching_rows = health_df.loc[(health_df['Country Name'] == country_name) & (health_df['Year'].isin(years))]
    for sub_idx, sub_row in matching_rows.iterrows():
        health_df.at[sub_idx, missing_col] = row[str(sub_row['Year'])]

In [24]:
# clean some all-zeros
print(len(health_df))
force_non_zero_cols = ['Population ages 15-19, female (% of female population)',
       'Population ages 15-19, male (% of male population)',
       'Population ages 20-24, female (% of female population)',
       'Population ages 20-24, male (% of male population)',
       'Population ages 25-29, female (% of female population)',
       'Population ages 25-29, male (% of male population)',
       'Population ages 30-34, female (% of female population)',
       'Population ages 30-34, male (% of male population)',
       'Population ages 35-39, female (% of female population)',
       'Population ages 35-39, male (% of male population)',
       'Population ages 40-44, female (% of female population)',
       'Population ages 40-44, male (% of male population)',
       'Population ages 45-49, female (% of female population)',
       'Population ages 45-49, male (% of male population)',
       'Population ages 50-54, female (% of female population)',
       'Population ages 50-54, male (% of male population)',
       'Population ages 55-59, female (% of female population)',
       'Population ages 55-59, male (% of male population)',
       'Population ages 60-64, female (% of female population)',
       'Population ages 60-64, male (% of male population)',
       'Population ages 65-69, female (% of female population)',
       'Population ages 65-69, male (% of male population)',
       'Population ages 70-74, female (% of female population)',
       'Population ages 70-74, male (% of male population)',
       'Population ages 75-79, female (% of female population)',
       'Population ages 75-79, male (% of male population)',
       'Population ages 80 and above, female (% of female population)',
       'Population ages 80 and above, male (% of male population)']
for col in force_non_zero_cols:
    health_df = health_df[health_df[col] != 0.0]
print(len(health_df))

11835
10472


In [25]:
# calculate % total for these ranges
for r in age_ranges:
    health_df['Population ages {} (% of total population)'.format(r)] = np.nan

for idx, row in health_df.iterrows():
    male = row['Population, male']
    female = row['Population, female']
    total = row['Population, total']
    if total == 0 or male == 0 or female == 0:
        continue
    for r in age_ranges:
        male_frac = row['Population ages {}, male (% of male population)'.format(r)]
        female_frac = row['Population ages {}, female (% of female population)'.format(r)]
        total_sub = (male_frac / 100.0 )* male + (female_frac / 100.0) * female
        total_frac = ((1.0 *total_sub) / total) * 100.0
        health_df.at[idx, 'Population ages {} (% of total population)'.format(r)] = total_frac

In [26]:
health_df.to_csv("health.csv", index=False)

In [13]:
health_df.columns

Index(['Country Name', 'Year',
       'Age dependency ratio (% of working-age population)',
       'Age dependency ratio, old (% of working-age population)',
       'Age dependency ratio, young (% of working-age population)',
       'Population ages 15-19, female (% of female population)',
       'Population ages 15-19, male (% of male population)',
       'Population ages 20-24, female (% of female population)',
       'Population ages 20-24, male (% of male population)',
       'Population ages 25-29, female (% of female population)',
       'Population ages 25-29, male (% of male population)',
       'Population ages 30-34, female (% of female population)',
       'Population ages 30-34, male (% of male population)',
       'Population ages 35-39, female (% of female population)',
       'Population ages 35-39, male (% of male population)',
       'Population ages 40-44, female (% of female population)',
       'Population ages 40-44, male (% of male population)',
       'Population

In [29]:
age_ranges_cp = age_ranges.copy()
age_ranges_cp.append('0-14')
print(age_ranges_cp)
for idx, row in health_df.iterrows():
    sum = 0
    for f in age_ranges_cp:
        sum += health_df.at[idx, 'Population ages {} (% of total population)'.format(f)]
    if sum < 95 or sum > 105:
        print(sum)
        male_sum = 0
        for f in age_ranges:
            male_sum += row['Population ages {}, male (% of male population)'.format(f)]
        female_sum = 0
        for f in age_ranges:
            female_sum += row['Population ages {}, female (% of female population)'.format(f)]    
        print("{} {} {} {}".format(row['Country Name'], row['Year'], male_sum, female_sum))    

['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80 and above', '0-14']
147.80243663068777
Yemen, Rep. 1990 93.55580227941822 98.85137001847137
94.6965603371722
Timor-Leste 2017 56.20709508162854 56.587274967164014
94.48218570343502
Timor-Leste 2000 49.48496989907535 49.740102841526735
94.70910329471678
Timor-Leste 2002 49.64026166668446 49.97441208189014
149.08339647348132
Samoa 1970 96.74456856966633 99.59980061240472
94.96389531564468
Syrian Arab Republic 2015 61.416502879574985 62.41173090006318
94.99983628687147
Syrian Arab Republic 2016 62.12686915491739 63.15603114390759
94.45929225845154
Timor-Leste 2001 49.47537231998098 49.777186640292804
